### Build a Simple LLM Application with LCEL(LangChain Expression Language)
LangChain Expression Language (LCEL) is an abstraction of some interesting Python concepts into a format that enables a "minimalist" code layer for building chains of LangChain components.

In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After this , you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"]=os.getenv("GEMINI_KEY")
#Langchain Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
#model=ChatGroq=(model="gemma2-9b-it")
model

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001D54B730050>, default_metadata=(), model_kwargs={})

In [3]:
from langchain_core.messages import HumanMessage,SystemMessage
messages=[
    SystemMessage(content="Translate the following from English To Hindi and provide alternative also:"),
    HumanMessage(content="Hii ,How are you ? I am here to answer all the queries that you have put in your email !! May i know in which mode of communcation you would like to communicate with us. we prefer communication over call coz we would be able to understand your queries in a better way,thanks")
]
result=model.invoke(messages)
result

AIMessage(content='Here are a few translations of the English text into Hindi, offering slightly different nuances:\n\n**Option 1 (Formal):**\n\nनमस्ते, आप कैसे हैं?  मैं आपके ईमेल में पूछे गए सभी प्रश्नों के उत्तर देने के लिए यहाँ हूँ!! क्या आप हमें बता सकते हैं कि आप किस माध्यम से हमारे साथ संवाद करना चाहेंगे? हम कॉल पर संवाद करना पसंद करते हैं क्योंकि इससे हम आपके प्रश्नों को बेहतर ढंग से समझ पाएँगे, धन्यवाद।\n\n*Namaste, aap kaise hain? Main aapke email mein puche gaye sabhi prashnon ke uttar dene ke liye yahan hun!! Kya aap hume bata sakte hain ki aap kis madhyam se hamare saath samvad karna chahenge? Ham call par samvad karna pasand karte hain kyunki isse hum aapke prashnon ko behtar dhang se samajh paenge, dhanyavaad.*\n\n\n**Option 2 (Slightly less formal):**\n\nनमस्ते, कैसे हैं आप?  मैं आपके ईमेल के सारे सवालों के जवाब देने आया हूँ!  आप किस तरह से हमसे बात करना चाहेंगे?  हम कॉल पर बात करना ज़्यादा पसंद करेंगे, क्योंकि इससे हम आपके सवाल अच्छे से समझ पाएँगे। धन्यवाद।\n\n*Namaste

Remember every model has it's on way to decide whether the whole output produced will lie in content or not

parser always return result.content i.e. content of ai message

so when we use groq model it will not give whole output inside content
and gemini model put everything inside conent

In [4]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few translations of the English text into Hindi, offering slightly different nuances:\n\n**Option 1 (Formal):**\n\nनमस्ते, आप कैसे हैं?  मैं आपके ईमेल में पूछे गए सभी प्रश्नों के उत्तर देने के लिए यहाँ हूँ!! क्या आप हमें बता सकते हैं कि आप किस माध्यम से हमारे साथ संवाद करना चाहेंगे? हम कॉल पर संवाद करना पसंद करते हैं क्योंकि इससे हम आपके प्रश्नों को बेहतर ढंग से समझ पाएँगे, धन्यवाद।\n\n*Namaste, aap kaise hain? Main aapke email mein puche gaye sabhi prashnon ke uttar dene ke liye yahan hun!! Kya aap hume bata sakte hain ki aap kis madhyam se hamare saath samvad karna chahenge? Ham call par samvad karna pasand karte hain kyunki isse hum aapke prashnon ko behtar dhang se samajh paenge, dhanyavaad.*\n\n\n**Option 2 (Slightly less formal):**\n\nनमस्ते, कैसे हैं आप?  मैं आपके ईमेल के सारे सवालों के जवाब देने आया हूँ!  आप किस तरह से हमसे बात करना चाहेंगे?  हम कॉल पर बात करना ज़्यादा पसंद करेंगे, क्योंकि इससे हम आपके सवाल अच्छे से समझ पाएँगे। धन्यवाद।\n\n*Namaste, kaise hain aap? 

Using LCEL to chain the components


In [5]:
chain=model|parser
chain.invoke(messages)

'Here are a few options for translating "Hii ,How are you ? I am here to answer all the queries that you have put in your email !! May i know in which mode of communcation you would like to communicate with us. we prefer communication over call coz we would be able to understand your queries in a better way,thanks" into Hindi, with slightly different nuances:\n\n\n**Option 1 (Formal):**\n\nनमस्ते, आप कैसे हैं?  मैं आपके ईमेल में पूछे गए सभी प्रश्नों के उत्तर देने के लिए यहाँ हूँ!! क्या आप हमें बता सकते हैं कि आप किस माध्यम से हमारे साथ संवाद करना चाहेंगे? हम कॉल पर संवाद करना पसंद करते हैं क्योंकि इससे हम आपके प्रश्नों को बेहतर ढंग से समझ पाएँगे, धन्यवाद।\n\n*Namaste, aap kaise hain? Main aapke email mein pooche gaye sabhi prashnon ke uttar dene ke liye yahan hoon!! Kya aap hume bata sakte hain ki aap kis madhyam se hamare saath samvad karna chahenge? Ham call par samvad karna pasand karte hain kyunki isse hum aapke prashnon ko behtar dhang se samajh paayenge, dhanyavaad.*\n\n\n**Optio

In [6]:
from langchain_core.prompts import ChatPromptTemplate
generic_template="translate the following into {language}:"
prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [7]:
result=prompt.invoke({"language":"Hindi","text":"hi ,Sachin Goyal this side ,it's interesting to learn and build gen ai projects"})
result.to_messages()

[SystemMessage(content='translate the following into Hindi:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi ,Sachin Goyal this side ,it's interesting to learn and build gen ai projects", additional_kwargs={}, response_metadata={})]

Chaining together using Lcel

In [8]:
chain=prompt|model|parser
chain.invoke({"language":"Hindi","text":"hi ,Sachin Goyal this side ,it's interesting to learn and build gen ai projects"})

'नमस्ते, मैं सचिन गोयल हूँ।  जेनरेटिव एआई प्रोजेक्ट्स सीखना और बनाना बहुत दिलचस्प है।'